In [1]:
import warnings
warnings.simplefilter('ignore')
import gc
import re
import time
import os
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy.stats import entropy, kurtosis
import xgboost as xgb
import seaborn as sns
from xgboost import plot_importance
import matplotlib.pyplot as plt
import pandas as pd
from math import *
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.datasets.samples_generator import make_blobs
import numpy as np
import lightgbm as lgb
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
# !pip install pyod
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF

# pd.set_option('display.max_columns', None)
# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)
pd.set_option('float_format', lambda x: '%.6f' % x)

%matplotlib inline

In [2]:
pathf = os.path.join("..", "data", "particles")

trainpd = pd.read_csv(os.path.join(pathf, "train.csv"))
print(trainpd.head(1))
trainshape = trainpd.shape
print(trainshape)
eventpd = pd.read_csv(os.path.join(pathf, "event.csv"))
print(eventpd.head(1))
print(eventpd.shape)
testpd = pd.read_csv(os.path.join(pathf, "test.csv"))
testshape = testpd.shape
print(testpd.head(1))
print(testpd.shape)


            x           y  z          t   terror        q  flag  event_id  \
0 -142.500000 -147.500000  0 767.879000 2.029660 1.050520     0         7   

   hit_id  
0       1  
(9473201, 9)
   event_id  nhit  nhitreal     energymc   thetamc     phimc       xcmc  \
0         7   426        70 48348.900000 63.168600 11.098200 -40.830000   

        ycmc  
0 114.030000  
(13315, 8)
            x           y  z          t   terror        q  event_id  hit_id
0 -142.500000 -127.500000  0 848.061000 1.998400 1.150670         9       1
(4086511, 8)


In [3]:
# (k(q,mc)*(t0+L))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+L) = (t+L)^2
# t0 方程 
# a = k(q,mc)^2
# b = 2*L*k(q,mc)^2 -dis*cos(phi)*sin(thmc)
# c = L^2 * k(q,mc)^2 + dis^2 - dis*cos(phi)*sin(thmc)*L - (t+L)^2 
# t0 = (-b +- (b^2 - 4*a*c)^(1/2))/2*a
data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')

data['fx'] = data['x'] - data['xcmc']
data['fy'] = data['y'] - data['ycmc']
data['phimc'] = data['phimc'] * np.pi / 180.
data['fphi'] = np.arctan2(data['fy'], data['fx']) - data['phimc']
data['fdis'] = np.sqrt(data['fx'] ** 2 + data['fy'] ** 2)
data['thetamc'] = data['thetamc'] * np.pi / 180.

data['fsinthmc'] = np.sin(data['thetamc'])
data['fsinthmc_v'] = 1.0/data['fsinthmc']
data['fcosphi'] = np.cos(data['fphi'])
data['fcosphi_v'] = 1.0/data['fcosphi']

data['fcosthmc'] = np.cos(data['thetamc'])
data['fcosthmc_v'] = 1.0/data['fcosthmc']
data['fsinphi'] = np.sin(data['fphi'])
data['fsinphi_v'] = 1.0/data['fsinphi']

data['ftanphi'] = np.tan(data['fphi'])
data['ftanphi_v'] = 1.0/data['ftanphi']
data['ftanthmc'] = np.tan(data['thetamc'])
data['ftanthmc_v'] = 1.0/data['ftanthmc']


# data['ft2'] = data['t'] ** 2
# data['fdis2'] = data['fdis'] ** 2

data['fttrue'] = data['t'] / data['terror']
data['terror_v'] = 1.0 / data['terror']
data['terror_v2'] =data['terror_v'] ** 2 
data['fttrue_v'] = 1.0 / data['fttrue']
data['fttrue2'] = data['fttrue'] ** 2
data['fttrue2_v'] = 1.0 / data['fttrue2'] 
data['nhitratio'] = data['nhit'] / data['nhitreal']
data['nhitratio_v'] = data['nhitratio']
data['energymc_v'] = 1.0 / data['energymc']
data['fenergymc2'] = data['energymc'] ** 2
data['fenergymc2_v'] = 1.0 / data['fenergymc2'] 
# data['q_v'] = 1.0 / data['q']
data['q2'] = data['q']
# data['q2_v'] = 1.0 / data['q2']

del data['fx']
del data['fy']
del data['x']
del data['y']
del data['z']
del data['xcmc']
del data['ycmc']
del data['fphi']
del data['phimc']
del data['nhitreal']
gc.collect()

11

In [4]:
info_new = pd.DataFrame()
info_new["event_id"] = data.groupby(["event_id"])["event_id"].mean()
info_new["fdis_mean"] = data.groupby(["event_id"])["fdis"].mean()
info_new["fdis_std"] = data.groupby(["event_id"])["fdis"].std()
info_new["fdis_stdmean"] = info_new["fdis_std"] / info_new["fdis_mean"]
info_new["ft_min"] = data.groupby(["event_id"])["t"].min()
info_new["ft_max"] = data.groupby(["event_id"])["t"].max()
info_new["t_mean"] = data.groupby(["event_id"])["t"].mean()
info_new["ft_std"] = data.groupby(["event_id"])["t"].std()
info_new["ft_stdmean"] = info_new["ft_std"] / info_new["t_mean"]
info_new["ft_mean"] = (info_new['t_mean']-info_new['ft_min']) / (info_new['ft_max']-info_new['ft_min'])
info_new.reset_index(drop=True, inplace=True)
data = pd.merge(data, info_new, on='event_id', how='left')

# data['ft_rel'] = (data['t']-data['ft_min']) / (data['ft_max']-data['ft_min'])
data['ft_rel'] = data['t'] / data['ft_std']
# data['ft_rel_std'] = data['ft_rel'] / data['ft_std']
data['ft2_rel'] = data['ft_rel'] ** 2
# data['ft2_rel_std'] = data['ft_rel_std'] ** 2
data['ft_rel_v'] = 1.0 / data['ft_rel']
data['ft2_rel_v'] = 1.0 / data['ft2_rel'] 
# (k(q,mc)*(t0+L))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+L) = (t+L)^2
data = data.sort_values(by=['event_id', 'ft_rel']).reset_index(drop=True)
# for i in [4, 6, 8, 10, 12]:
for i in [7, 8, 11,17, 47]:
    data[f'ft_{i}diff'] = data.groupby('event_id')['ft_rel'].diff(periods=i).fillna(0)
    
data['fdis_rel'] = data['fdis'] / data['fdis_mean']
data['fdis_rel_std'] = data['fdis_rel'] / data['fdis_std']
data['fdis2_rel'] = data['fdis_rel'] ** 2
data['fdis2_rel_std'] = data['fdis_rel_std'] ** 2

data['fdis_t_rel'] = data['fdis_rel'] / data['ft_rel']
data['fdis_tmean_rel'] = data['fdis_rel'] / data['ft_mean']
data['fdis_t_rel'].fillna(0, inplace=True)
# data['fdis-t_rel'] = data['fdis_rel'] * data['ft_rel']
# data['fdis-tmean_rel'] = data['fdis_rel'] * data['ft_mean']
# data['fdis_t_normal'] = data['fdis_rel'] / data['ft_normal']
# data['fdis_t_normal'].fillna(0, inplace=True)
# data['fdis-t_normal'] = data['fdis_rel'] * data['ft_normal']

# data = data.sort_values(by=['event_id', 'fdis_rel']).reset_index(drop=True)
# for i in [5, 6, 7, 8, 10, 11,12]:
#     data[f'fdis_rel_{i}diff'] = data.groupby('event_id')['fdis_rel'].diff(periods=i).fillna(0)
data['fx_normal'] = data['fcosphi'] * data['fdis_rel']
# data['fy_normal'] = data['fsinphi'] * data['fdis_rel']

data['fcossin'] = data['fcosphi'] * data['fsinthmc']
data['fdis_relcossin'] =  data['fdis_rel'] * data['fcossin']
data['fdis2_relcossin'] = data['fdis2_rel'] * data['fcossin']
# data['fdis2_rel_stdcossin'] = data['fdis2_rel_std'] * data['fcossin']
data['ft2_relcossin'] = data['ft2_rel'] * data['fcossin']

data['fsincos'] = data['fsinphi'] * data['fcosthmc']
# data['fdis_relsincos'] = data['fdis_rel'] * data['fsincos']
# data['fdis2_relsincos'] = data['fdis2_rel'] * data['fsincos']
# data['fdis2_rel_stdsincos'] = data['fdis2_rel_std'] * data['fsincos']
# data['ft_relsincos'] =  data['ft_rel'] * data['fsincos']
# data['ft2_relsincos'] = data['ft2_rel'] * data['fsincos']

data['fsinsin'] = data['fsinphi'] * data['fsinthmc']
# data['fdis_relsinsin'] =  data['fdis_rel'] * data['fsinsin']
# data['fdis2_relsinsin'] = data['fdis2_rel'] * data['fsinsin']
# data['fdis2_rel_stdsinsin'] = data['fdis2_rel_std'] * data['fsinsin']
# data['ft_relsinsin'] =  data['ft_rel'] * data['fsinsin']
# data['ft2_relsinsin'] = data['ft2_rel'] * data['fsinsin']

data['fcoscos'] = data['fcosphi'] * data['fcosthmc']

data['ft_relsinsin'] = data['ft_rel'] * data['fsinsin']
data['ft_relcoscos'] = data['ft_rel'] * data['fcoscos']
data['ft_relsincos'] = data['ft_rel'] * data['fsincos']
data['ft_relcossin'] = data['ft_rel'] * data['fcossin']

data['ft_rel_errcoscos'] = data['ft_relcoscos'] * data['terror']
data['fdis2_rel_stdcoscos'] = data['fdis2_rel_std'] * data['fcoscos']
data['ft2_relcoscos'] = data['ft2_rel'] * data['fcoscos']

data['fdis_t_relcossin'] = data['fdis_relcossin'] / data['ft_rel']
data['fdis_t_relcossin'].fillna(0, inplace=True)
data['fdis-t_relcossin'] = data['fdis_relcossin'] * data['ft_rel']

# # 要保留
# del data['ft_7diff']
# del data['ft_8diff']
# del data['ft_11diff']
# del data['ft_17diff']
# del data['fdis_t_rel']
# del data['fttrue2']
# del data['ft_rel_errcoscos']
# del data['fdis_t_relcossin']
# del data['fdis-t_relcossin']
# del data['q2']
# del data['q']
# del data['ft_47diff']
# del data['ft2_rel_v']
# del data['fttrue2_v']
# del data['fttrue_v']
# del data['fsinthmc_v']
# del data['ft_rel']

del data['t']
del data['ft_relcossin']
del data['ft2_rel']
del data['ft_relcoscos']
del data['ft_rel_v']
del data['fcosphi_v']
del data['ft2_relcossin']
del data['ft2_relcoscos']
del data['ftanphi']
del data['ft_relsincos']
del data['ft_relsinsin']
del data['fttrue']
del data['ftanphi_v']
del data['fsinphi_v']

# 要保留但不明显
del data['fdis']
del data['fdis2_rel']
del data['fdis_tmean_rel']
del data['terror_v2']
del data['nhitratio']
del data['fdis2_relcossin']
del data['fdis2_rel_stdcoscos']
del data['ftanthmc']
del data['energymc_v']
del data['nhitratio_v']
del data['fdis_rel_std']
del data['fenergymc2']
del data['fdis2_rel_std']
del data['fdis_stdmean']

# 真删除
del data['fx_normal']
del data['fcossin']
del data['fdis_relcossin']
del data['terror_v']
del data['ft_std']
del data['fdis_rel']
del data['ftanthmc_v']
del data['fenergymc2_v']
del data['fcoscos']
del data['fcosthmc_v']
del data['fdis_mean']
del data['ft_mean']
del data['nhit']
del data['fdis_std']
del data['t_mean']
del data['energymc']
del data['terror']
del data['ft_stdmean']
del data['ft_min']
del data['ft_max']
del data['fcosphi']
del data['thetamc']
del data['fsinthmc']
del data['fcosthmc']
del data['fsincos']
del data['fsinsin']
del data['fsinphi']

gc.collect()

0

In [5]:
print(trainshape[0])
print(data.shape)
testpd = data[data.flag.isna()].reset_index()
# data.loc[data.flag.isna() & (data.t < -900), 'flag'] = 0
# data.loc[data.flag.isna() & ((data.t > 1850) | (data.q < 0)), 'flag'] = 1
trainpd = data[data.flag.notna()].reset_index()
trainpd['flag'] = trainpd['flag'].astype('int')
# trainpd = data[:trainshape[0]].reset_index()
# testpd = data[trainshape[0]:].reset_index()
del data
gc.collect()

9473201
(13559712, 20)


0

In [6]:
print(trainpd.columns)
# feature = [x for x in trainpd.columns if x not in ['flag', 'index', 'hit_id', 'event_id']]
feature = [x for x in trainpd.columns if x not in ['flag', 'index', 'event_id']]
labels = trainpd[['flag']]

scaler = MinMaxScaler(feature_range=(0, 1))
trainpd[feature] = scaler.fit_transform(trainpd[feature])

del trainpd['flag']
del testpd['flag']

X = np.concatenate((trainpd,labels),axis=1)

Index(['index', 'q', 'flag', 'event_id', 'hit_id', 'fsinthmc_v', 'fttrue_v',
       'fttrue2', 'fttrue2_v', 'q2', 'ft_rel', 'ft2_rel_v', 'ft_7diff',
       'ft_8diff', 'ft_11diff', 'ft_17diff', 'ft_47diff', 'fdis_t_rel',
       'ft_rel_errcoscos', 'fdis_t_relcossin', 'fdis-t_relcossin'],
      dtype='object')


In [7]:
random_state = np.random.RandomState(42)
outliers_fraction = 0.05
# Define seven outlier detection tools to be compared
classifiers = {
        'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor (CBLOF)':CBLOF(contamination=outliers_fraction,check_estimator=False, random_state=random_state),
        'Feature Bagging':FeatureBagging(LOF(n_neighbors=35),contamination=outliers_fraction,check_estimator=False,random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Average KNN': KNN(method='mean',contamination=outliers_fraction)
}


In [ ]:
dfx = pd.DataFrame()
for i, (clf_name, clf) in enumerate(classifiers.items()):
    start_time = time.time()
    print(clf_name)
    clf.fit(X)
    # predict raw anomaly score
    scores_pred = clf.decision_function(X) * -1
        
    # prediction of a datapoint category outlier or inlier
    y_pred = clf.predict(X)
    used_time = (time.time() - start_time) / 3600
    print(f'used_time: {used_time:.2f} hours')
#     n_inliers = len(y_pred) - np.count_nonzero(y_pred)
#     n_outliers = np.count_nonzero(y_pred == 1)
#     plt.figure(figsize=(10, 10))
    
    # copy of dataframe
    dfx[clf_name] = y_pred
    
    # IX1 - inlier feature 1,  IX2 - inlier feature 2
    

Angle-based Outlier Detector (ABOD)


In [ ]:
#阈值大概在0.2-0.4之间 本题对召回率较敏感，可适当降低一下阈值
thre = 0.25

#生成提交文件
sub = pd.DataFrame()
sub['hit_id'] = testpd['hit_id']
sub['flag_pred'] = fy_submission
sub['event_id'] = testpd['event_id']
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= thre else 0)
sub.to_csv(os.path.join(pathf, "subsample.csv").format(sub['flag_pred'].mean()), index=False)

In [ ]:
# data.plot(x='fcosphi_v', y='flag', kind="scatter")